In [66]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from interpolation import CensusData, sequences, BlockInterpolator, CentroidInterpolator, archive, interpolation
from sklearn.pipeline import Pipeline
from category_encoders.target_encoder import TargetEncoder
from xgboost import XGBClassifier, plot_tree
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import make_column_transformer
import tensorflow
import keras
from keras.layers import Dense, Input
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
#Change these values to rerun the notebook
filled_1850 = pd.read_csv("../../data/dwelling_filled_sum_1850_mn_v01.csv")
cd_1850 = pd.read_csv("../../data/cd_1850_XYupdate.csv") #For calculating centroids
filled_1850.rename(columns = {"CENSUS_WARD_NUM":"Ward_Num", "CENSUS_SERIALP":"CENSUS_SERIALB", "BLOCK_NUM":"block_num", "CENSUS_REEL_HOUSEHOLD":"CENSUS_REEL", "CENSUS_SEX":"CENSUS_SEXB", "CENSUS_PAGENO_HOUSEHOLD":"CENSUS_PAGENUM", "CENSUS_RACE":"CENSUS_RACEB", "CD_X":"cd_X", "CD_Y":"cd_Y"}, inplace = True)
ward_col = "Ward_Num"
dwelling_col = "dwelling_id"
dwelling_col_num = "CENSUS_DWELLING_NUM"
block_col = "block_num"
x_col = "cd_X"
y_col = "cd_Y"

In [14]:
census_data = CensusData(filled_1850, ward_col=ward_col, dwelling_col=dwelling_col, block_col =  block_col, x_col = x_col, y_col = y_col)
census_data.apply_sequencing(fixed = False)

#### Neural Networks

In [27]:
all_cols = ["CENSUS_SERIALB", "CENSUS_AGE", "CENSUS_PAGENUM", "CENSUS_SEXB", "CENSUS_RACEB","CENSUS_DWELLING_SIZE","CENSUS_SEQ_NUM", "CENSUS_LABFORCE", "CENSUS_LINE", "CENSUS_MARST", "CENSUS_NAMEFRST", "CENSUS_NAMELAST", "CENSUS_IMPREL",'CENSUS_DWELLING_SEQ', "sequence_order_enum", "sequence_id", "dwelling_seq_id"]

In [28]:
block_centroids = {ward:{block:interpolation.make_centroid(df_block[x_col], df_block[y_col]) for block, df_block in df.groupby(block_col)} for ward,df in cd_1850.groupby(ward_col)}

In [69]:
model = Sequential()
model.add(Dense (256, input_dim=4853, activation = "relu"))
model.add(Dense (128, activation = "relu"))
model.add(Dense (5, activation = "sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 256)               1242624   
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_23 (Dense)             (None, 5)                 645       
Total params: 1,276,165
Trainable params: 1,276,165
Non-trainable params: 0
_________________________________________________________________


In [71]:
interpolate_dense = CentroidInterpolator(census_data, 10, model, all_cols, AgglomerativeClustering(5), block_centroids)
interpolate_dense.apply_clustering()
train, test = interpolate_dense.stratified_train_test()
train_ohe = OneHotEncoder().fit_transform(train.loc[:,all_cols].fillna(0))

In [72]:
labels = to_categorical(np.array(train["cluster"]))

In [73]:
interpolate_dense.model.fit(train_ohe, labels, epochs=150, batch_size=10)

Epoch 1/150
4586/4586 [==============================] - 10s 2ms/step - loss: 0.1210 - accuracy: 0.9536
Epoch 2/150
4586/4586 [==============================] - 9s 2ms/step - loss: 5.1868e-04 - accuracy: 1.0000
Epoch 3/150
4586/4586 [==============================] - 9s 2ms/step - loss: 1.1928e-04 - accuracy: 1.0000
Epoch 4/150
4586/4586 [==============================] - 9s 2ms/step - loss: 5.6289e-05 - accuracy: 1.0000
Epoch 5/150
1750/4586 [==========>...................] - ETA: 7s - loss: 3.4601e-05 - accuracy: 1.0000

KeyboardInterrupt: 

In [76]:
print(interpolate_dense.model.evaluate(train_ohe, labels))

4586/4586 [==============================] - 1s 136us/step
[3.168059396164043e-05, 1.0]


In [79]:
test_ohe = OneHotEncoder().fit_transform(test.loc[:,all_cols].fillna(0))
test_labels = to_categorical(np.array(test["cluster"]))

In [83]:
train_ohe.shape

(4586, 4853)

In [84]:
test_ohe.shape

(1347, 1823)

In [81]:
print(interpolate_dense.model.evaluate(test_ohe, test_labels))

ValueError: Error when checking input: expected dense_21_input to have shape (4853,) but got array with shape (1823,)

#### XGBoost with tuning